<a href="https://colab.research.google.com/github/MuhammadIrzam447/MultiModel/blob/master/Train_30.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# New Section

In [ ]:
import random
from datasets import Dataset
from torch.utils.data import Dataset, DataLoader
device = "cuda" if torch.cuda.is_available() else "cpu"


In [ ]:
from datasets import load_dataset

ds = load_dataset("imagefolder", data_dir="/content/Dataset(s)/jointHF-train+test_unseen/train", split="train")

In [ ]:
import os
def filter_funtion(example):
    img = example["image"]
    filename = os.path.basename(img.filename)
    return filename.endswith("_4.png")

def filter_funtion1(example):
    img = example["image"]
    filename = os.path.basename(img.filename)
    return filename.endswith("_3.png")

In [ ]:
ds_imgs = ds.filter(filter_funtion)

In [ ]:
filtered_ds = ds.filter(filter_funtion1)

In [ ]:
total_samples = len(filtered_ds)
desired_samples = int(total_samples * 0.3)

ds_encoded = filtered_ds.shuffle(seed=42).select([i for i in range(desired_samples)])

In [ ]:
ds_encoded

In [ ]:
from datasets import concatenate_datasets
ds_train = concatenate_datasets([ds_imgs, ds_encoded])

In [ ]:
ds_train

In [ ]:
from torchvision import transforms
from PIL import Image

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


def apply_transform(example):
    image = Image.open(example["image"].filename).convert("RGB")  # Ensure the image is in RGB mode
    image = transform(image)
    return {"image": image, "label": example["label"]}

In [ ]:
ds_transformed = ds_train.map(apply_transform)

In [ ]:
batch_size = 32
train_loader = DataLoader(ds_train, batch_size=batch_size, shuffle=True)

In [ ]:
labels = ds_train.features["label"].names
labels

In [ ]:
num_classes = len(labels)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models

resnet = models.resnet101(pretrained=True)
num_features = resnet.fc.in_features
resnet.fc = nn.Linear(num_features, num_classes)  # num_classes is the number of classes in your dataset
resnet.to(device)
print(resnet)


In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(resnet.parameters(), lr=0.001, momentum=0.9)
num_epochs = 20

In [ ]:
training_loss = []

for epoch in range(num_epochs):
    # Set the model to training mode
    resnet.train()
    running_loss = 0.0

    # Iterate over the data loader
    for images, labels in train_loader:
        # Move the images and labels to the GPU if available
        images = images.to(device)
        labels = labels.to(device)

        # Clear the gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = resnet(images)

        # Compute the loss
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        # Update the running loss
        running_loss += loss.item() * images.size(0)

    # Calculate the average loss for the epoch
    epoch_loss = running_loss / len(data_loader.dataset)
    training_loss.append(epoch_loss)
    # Print the epoch loss
    print(f"Epoch {epoch+1}/{num_epochs} Training Loss: {epoch_loss:.4f}")

# New Section

In [ ]:
# import os

# # Define the path to your dataset folder
# dataset_folder = '/content/Dataset(s)/jointHF-train+test_unseen/test'

# # Define the path for the output text file
# output_file = 'test_dataset(2).txt'

# # Open the output file for writing
# with open(output_file, 'w') as f:
#     # Iterate through the subdirectories in the dataset folder
#     for label in os.listdir(dataset_folder):
#         label_path = os.path.join(dataset_folder, label)

#         # Check if it's a directory (class folder)
#         if os.path.isdir(label_path):
#             # Get the label (class name) from the folder name
#             class_name = label

#             # Iterate through the images in the class folder
#             for image_file in os.listdir(label_path):
#                 image_path = os.path.join(label_path, image_file)

#                 # Write the image path and label to the output file
#                 f.write(f'{image_path}, {class_name}\n')


Train Dataset

In [ ]:
import os

image_file_paths = []
image_genre_labels = []
enc_file_paths = []
enc_genre_labels = []

labels_file = "/content/Dataset(s)/jointHF-train+test_unseen/train_dataset(2).txt"

with open(labels_file, 'r') as file:
    for line in file:
        parts = line.strip().split(', ')
        filename = parts[0].strip()
        labels = parts[1].strip()

        if filename.endswith("_3.png"):
            enc_file_paths.append(filename)
            enc_genre_labels.append(labels)

        elif filename.endswith("_4.png"):
            image_file_paths.append(filename)
            image_genre_labels.append(labels)

In [ ]:
len(image_file_paths), len(enc_file_paths)

(9500, 9500)

In [ ]:
image_file_paths[0:5] , image_genre_labels[0:5]

(['/content/Dataset(s)/jointHF-train+test_unseen/train/1/10547.jpg_4.png',
  '/content/Dataset(s)/jointHF-train+test_unseen/train/1/29380.jpg_4.png',
  '/content/Dataset(s)/jointHF-train+test_unseen/train/1/53927.jpg_4.png',
  '/content/Dataset(s)/jointHF-train+test_unseen/train/1/23485.jpg_4.png',
  '/content/Dataset(s)/jointHF-train+test_unseen/train/1/26495.jpg_4.png'],
 ['1', '1', '1', '1', '1'])

In [ ]:
enc_file_paths[0:5] , enc_genre_labels[0:5]

(['/content/Dataset(s)/jointHF-train+test_unseen/train/1/70623.jpg_3.png',
  '/content/Dataset(s)/jointHF-train+test_unseen/train/1/25314.jpg_3.png',
  '/content/Dataset(s)/jointHF-train+test_unseen/train/1/08637.jpg_3.png',
  '/content/Dataset(s)/jointHF-train+test_unseen/train/1/72509.jpg_3.png',
  '/content/Dataset(s)/jointHF-train+test_unseen/train/1/45802.jpg_3.png'],
 ['1', '1', '1', '1', '1'])

In [ ]:
unique_labels = set(enc_genre_labels)
unique_labels_list = list(unique_labels)

print("Unique Labels:", unique_labels_list)

Unique Labels: ['1', '0']


In [ ]:
num_classes = len(unique_labels_list)

In [ ]:
import random


percentage = 0.3

label_counts = {label: 0 for label in unique_labels_list}

num_samples_to_include_per_label = {}
for label in unique_labels_list:
    label_count = enc_genre_labels.count(label)
    num_samples_to_include = int(label_count * percentage)
    num_samples_to_include_per_label[label] = num_samples_to_include


filtered_enc_file_paths = []
filtered_enc_genre_labels = []


selected_samples_per_label = {label: 0 for label in unique_labels_list}
for path, label in zip(enc_file_paths, enc_genre_labels):
    if selected_samples_per_label[label] < num_samples_to_include_per_label[label]:
        filtered_enc_file_paths.append(path)
        filtered_enc_genre_labels.append(label)
        selected_samples_per_label[label] += 1

In [ ]:
len(filtered_enc_file_paths)

2850

In [ ]:
# Create new lists to combine data from both lists
train_file_paths = enc_file_paths + filtered_enc_file_paths
train_genre_labels = enc_genre_labels + filtered_enc_genre_labels

In [ ]:
len(train_file_paths)

12350

In [ ]:
train_hot_labels = []

for labels in train_genre_labels:
    uni_hot = [1 if label in labels else 0 for label in unique_labels_list]
    train_hot_labels.append(uni_hot)

In [ ]:
len(train_hot_labels)

In [ ]:
train_hot_labels[0:5]

Test Dataset

In [ ]:
import os

test_image_file_paths = []
test_image_genre_labels = []
test_enc_file_paths = []
test_enc_genre_labels = []

test_labels_file = "/content/Dataset(s)/jointHF-train+test_unseen/test_dataset(2).txt"

with open(test_labels_file, 'r') as file:
    for line in file:
        parts = line.strip().split(', ')
        filename = parts[0].strip()
        labels = parts[1].strip()

        if filename.endswith("_3.png"):
            test_enc_file_paths.append(filename)
            test_enc_genre_labels.append(labels)

        elif filename.endswith("_4.png"):
            test_image_file_paths.append(filename)
            test_image_genre_labels.append(labels)

In [ ]:
len(test_image_file_paths), len(test_enc_file_paths)

(2000, 2000)

In [ ]:
test_image_file_paths[0:5] , test_image_genre_labels[0:5]

(['/content/Dataset(s)/jointHF-train+test_unseen/test/1/95876.jpg_4.png',
  '/content/Dataset(s)/jointHF-train+test_unseen/test/1/65803.jpg_4.png',
  '/content/Dataset(s)/jointHF-train+test_unseen/test/1/02564.jpg_4.png',
  '/content/Dataset(s)/jointHF-train+test_unseen/test/1/81329.jpg_4.png',
  '/content/Dataset(s)/jointHF-train+test_unseen/test/1/59814.jpg_4.png'],
 ['1', '1', '1', '1', '1'])

In [ ]:
import random

label_counts = {label: 0 for label in unique_labels_list}

test_num_samples_to_include_per_label = {}
for label in unique_labels_list:
    label_count = test_enc_genre_labels.count(label)
    num_samples_to_include = int(label_count * percentage)
    test_num_samples_to_include_per_label[label] = num_samples_to_include


test_filtered_enc_file_paths = []
test_filtered_enc_genre_labels = []


test_selected_samples_per_label = {label: 0 for label in unique_labels_list}
for path, label in zip(test_enc_file_paths, test_enc_genre_labels):
    if test_selected_samples_per_label[label] < test_num_samples_to_include_per_label[label]:
        test_filtered_enc_file_paths.append(path)
        test_filtered_enc_genre_labels.append(label)
        test_selected_samples_per_label[label] += 1

In [ ]:
len(test_filtered_enc_file_paths)

600

In [ ]:
# Create new lists to combine data from both lists
test_file_paths = test_enc_file_paths + test_filtered_enc_file_paths
test_genre_labels = test_enc_genre_labels + test_filtered_enc_genre_labels

In [ ]:
len(test_file_paths)

2600

In [ ]:
test_hot_labels = []

for labels in test_genre_labels:
    uni_hot = [1 if label in labels else 0 for label in unique_labels_list]
    test_hot_labels.append(uni_hot)

In [ ]:
len(test_hot_labels)

In [ ]:
test_hot_labels[0:5]

Use Dataset

In [ ]:
from torch.utils.data import Dataset
from torchvision import transforms
from PIL import Image

class CustomDataset(Dataset):
    def __init__(self, file_paths, labels, transform=None):
        self.file_paths = file_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.file_paths)

    def __getitem__(self, idx):
        img_path = self.file_paths[idx]
        image = Image.open(img_path)
        label = int(self.labels[idx])
        # label = torch.tensor(label,dtype=torch.float32)

        if self.transform:
            image = self.transform(image)

        return image, label


transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [ ]:
train_dataset = CustomDataset(train_file_paths, train_genre_labels, transform=transform)
test_dataset = CustomDataset(test_file_paths, test_genre_labels, transform=transform)

In [ ]:
from torch.utils.data import DataLoader

batch_size = 32  # Adjust as needed
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models
device = "cuda" if torch.cuda.is_available() else "cpu"

resnet = models.resnet101(pretrained=True)
num_features = resnet.fc.in_features
resnet.fc = nn.Linear(num_features, num_classes)  # num_classes is the number of classes in your dataset
resnet.to(device)
print(resnet)


In [ ]:
criterion = nn.CrossEntropyLoss()
# optimizer = optim.SGD(resnet.parameters(), lr=0.001, momentum=0.9)
optimizer = optim.Adam(resnet.parameters(), lr=0.001)
num_epochs = 20

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, roc_auc_score
import numpy as np

In [32]:
training_loss = []

for epoch in range(num_epochs):
    resnet.train()
    running_loss = 0.0


    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = resnet(images)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)
    training_loss.append(epoch_loss)

    print("Training Loss==========================>>")
    print(f"Epoch {epoch+1}/{num_epochs} Training Loss: {epoch_loss:.4f}")

    resnet.eval()
    predicted_classes = []
    actual_labels = []
    with torch.no_grad():
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = resnet(images)

            _, predicted_label = torch.max(outputs, 1)

            predicted_classes.extend(predicted_label.cpu().tolist())
            actual_labels.extend(labels.cpu().tolist())

    accuracy = accuracy_score(actual_labels, predicted_classes)
    # precision = precision_score(actual_labels, predicted_classes, average='weighted')
    # recall = recall_score(actual_labels, predicted_classes, average='weighted')
    # f1 = f1_score(actual_labels, predicted_classes, average='weighted')

    print("Accuracy:", accuracy)
    # print("Precision:", precision)
    # print("Recall:", recall)
    # print("F1-score:", f1)

    print(classification_report(actual_labels, predicted_classes))
    # print("Confusion Matrix: ")
    # cm = confusion_matrix(actual_labels, predicted_classes)
    # print(cm)
    predicted_classes = np.array(predicted_classes)
    actual_labels = np.array(actual_labels)
    print(roc_auc_score(actual_labels, predicted_classes))


    save_dir = "/content/Model/Models-Train-27/"
    os.makedirs(save_dir, exist_ok=True)  # Create the directory if it doesn't exist

    model_name = str(epoch+1) + "_model.pth"
    save_path = os.path.join(save_dir, model_name)  # Specify the complete path to the model file
    torch.save(resnet.state_dict(), save_path)

Training Loss==========================>>
Epoch 1/20 Training Loss: 0.6645
Accuracy: 0.6115384615384616
              precision    recall  f1-score   support

           0       0.64      0.86      0.73      1625
           1       0.46      0.20      0.28       975

    accuracy                           0.61      2600
   macro avg       0.55      0.53      0.51      2600
weighted avg       0.57      0.61      0.56      2600

0.5298461538461539
Training Loss==========================>>
Epoch 2/20 Training Loss: 0.6467
Accuracy: 0.6126923076923076
              precision    recall  f1-score   support

           0       0.63      0.90      0.74      1625
           1       0.45      0.14      0.21       975

    accuracy                           0.61      2600
   macro avg       0.54      0.52      0.48      2600
weighted avg       0.56      0.61      0.54      2600

0.5178461538461538
Training Loss==========================>>
Epoch 3/20 Training Loss: 0.6421
Accuracy: 0.5957692307692